In [1]:
import torch
import torch.nn as nn
import numpy as np
import torchvision

In [2]:
from torchvision.models import vgg16, VGG16_Weights

In [3]:
%cd /kaggle/input/aidtransformer

/kaggle/input/aidtransformer


# Load Data

In [4]:
pip install /kaggle/input/dehazingimagesnatsort/natsort-8.4.0-py3-none-any.whl

Processing /kaggle/input/dehazingimagesnatsort/natsort-8.4.0-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [5]:
from dataset import *
from torch.utils.data import DataLoader

In [6]:
Dataset = '/kaggle/input/dehazingimagesdata/Dataset/Training_data/RICE'

In [7]:
# Create the data loader
data_loader = DataLoaderTrain(rgb_dir=Dataset, img_options={'patch_size': 256})

# Create the data loader
# data_loader = DataLoader(dataset, batch_size=16, shuffle=True)

In [8]:
# import cv2
# from google.colab.patches import cv2_imshow

# path = '/content/drive/MyDrive/AIDTransformer/testing_data/RICE/input/0.png'

# img = cv2.imread(path)
# img.shape
# #cv2_imshow(img)

In [9]:
data_loader
len(data_loader)
print(len(data_loader[499]))

torch.Size([3, 256, 256])
4


# Model

In [10]:
!pip install timm

In [11]:
!pip install /kaggle/input/dehazingimageeinops/einops-0.7.0-py3-none-any.whl

Processing /kaggle/input/dehazingimageeinops/einops-0.7.0-py3-none-any.whl


In [12]:
from model import *

In [13]:
model = Network()

/opt/conda/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


# perpectual loss

In [14]:
use_gpu = torch.cuda.is_available()
if use_gpu:
    print("Using CUDA")
else: 
    print("using CPU")

Using CUDA


In [15]:
vgg_model = vgg16(weights=VGG16_Weights.DEFAULT)

if torch.cuda.is_available():
    vgg_model.cuda()

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 214MB/s]


In [16]:
from collections import namedtuple

In [17]:
LossOutput = namedtuple("LossOutput", ["relu1_2", "relu2_2", "relu3_3"])

class LossNetwork(torch.nn.Module):
    def __init__(self, vgg_model):
        super(LossNetwork, self).__init__()
        self.vgg_layers = vgg_model.features
        self.layer_name_mapping = {
            '3': "relu1_2",
            '8': "relu2_2",
            '15': "relu3_3"
        }

    def forward(self, x):
        output = {}
        for name, module in self.vgg_layers._modules.items():
            x = module(x)
            if name in self.layer_name_mapping:
                output[self.layer_name_mapping[name]] = x
        return LossOutput(**output)


In [18]:
# Assuming output and ground_truth are your output and ground truth images
loss_network = LossNetwork(vgg_model)
loss_network.eval()
def perpectual_loss(y_pred, y_train):
  output_features = loss_network(y_pred)
  ground_truth_features = loss_network(y_train)

  # Calculate the perceptual loss
  perceptual_loss = 0
  for output_feature, ground_truth_feature in zip(output_features, ground_truth_features):
      perceptual_loss += torch.nn.functional.l1_loss(output_feature, ground_truth_feature)
  return perceptual_loss

# edge loss

In [19]:
import cv2 as cv

In [20]:
def sobel_edge_loss(img):
    #print('sobel')
    print(img.shape)
    #print(type(img))
    #img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    sobel_kernel_x = torch.tensor([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=torch.float32).unsqueeze(0).unsqueeze(0).expand(-1, 3, -1, -1)
    sobel_kernel_y = torch.tensor([[1, 2, 1], [0, 0, 0], [-1, -2, -1]], dtype=torch.float32).unsqueeze(0).unsqueeze(0).expand(-1, 3, -1, -1)

    sobel_kernel_x = sobel_kernel_x.to(img.device)
    sobel_kernel_y = sobel_kernel_y.to(img.device)

    edges_x = F.conv2d(img, sobel_kernel_x, padding=0)
    edges_y = F.conv2d(img, sobel_kernel_y, padding=0)

    edges = torch.sqrt(edges_x**2 + edges_y**2)

    return edges

In [21]:
def edge_loss(output, target):

    output_edges = sobel_edge_loss(output)
    target_edges = sobel_edge_loss(target)

    loss = F.l1_loss(output_edges, target_edges)

    return loss

# L1 Loss và Total Loss

In [22]:
import torch.nn.functional as F

In [23]:
L1_Loss = nn.L1Loss()
def loss_function(y_pred, y_train):
  loss1 = L1_Loss(y_pred, y_train)
  loss2 = edge_loss(y_pred, y_train)
  loss3 = perpectual_loss(y_pred, y_train)

  total_loss = 1*loss1 + 5*loss2 + 10*loss3
  return total_loss

# Training

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Network(
  embed_dim=32, token_projection=conv, token_mlp=ffn,win_size=8
  (pos_drop): Dropout(p=0.0, inplace=False)
  (input_proj): InputProj(
    (proj): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.01, inplace=True)
    )
  )
  (output_proj): OutputProj(
    (proj): Sequential(
      (0): Conv2d(96, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
  (output_proj1): OutputProj(
    (proj): Sequential(
      (0): Conv2d(32, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
  (encoderlayer_0): TRANSFORMER_BLOCK(
    dim=32, input_resolution=(128, 128), depth=2
    (blocks): ModuleList(
      (0): Deformable_Attentive_Transformer(
        dim=32, input_resolution=(128, 128), num_heads=1, win_size=8, shift_size=0, mlp_ratio=4.0
        (norm1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        (attn): WindowAttention(
          dim=32, win_size=(8, 8), num_heads=1
    

In [25]:
!pip install tqdm

In [26]:
from tqdm import tqdm
import math

In [27]:
def expand2square(timg,factor=32.0):
    timg = timg.unsqueeze(0)
    _, _, h, w = timg.size()

    X = int(math.ceil(max(h,w)/float(factor))*factor)

    img = torch.zeros(1,3,X,X).type_as(timg) # 3, h,w
    mask = torch.zeros(1,1,X,X).type_as(timg)


    img[:,:, ((X - h)//2):((X - h)//2 + h),((X - w)//2):((X - w)//2 + w)] = timg
    mask[:,:, ((X - h)//2):((X - h)//2 + h),((X - w)//2):((X - w)//2 + w)].fill_(1.0)

    return img, mask

In [28]:
#print(data_loader[0])

In [29]:
# data_loader[0][0]

In [30]:
from torch.optim.lr_scheduler import CosineAnnealingLR

In [31]:
# checkpoint = torch.load('/content/drive/MyDrive/AIDTransformer/checkpoint.pth')
# model.load_state_dict(checkpoint['state_dict'])
# optimizer = torch.optim.Adam(model.parameters(), lr = 0.0002)
# optimizer.load_state_dict(checkpoint['optimizer'])
# epoch = checkpoint['epoch']
# scheduler = CosineAnnealingLR(optimizer, T_max = 100, verbose = True)

In [32]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0002)
scheduler = CosineAnnealingLR(optimizer, T_max = 100, verbose = True)

epochs = 2
#final_loss = []
for i in range(epochs):
  #for ii in range(len(data_loader)):
  for ii, data_train in enumerate(tqdm(data_loader), 0):
    print(i, ii)
    rgb_gt = data_train[0].unsqueeze(0).to('cuda')

    rgb_noisy, mask = expand2square(data_train[1].cuda(), factor=128)
    rgb_noisy, mask = rgb_noisy.to('cuda'), mask.to('cuda')

    optimizer.zero_grad()

    # print(rgb_gt)
    # Forward pass
    outputs = model(rgb_noisy)
    #outputs = model(rgb_noisy, 1 - mask)

    # Compute loss
    loss = loss_function(outputs, rgb_gt)
    # final_loss.append(loss)

    # Backward pass and optimize
    loss.backward()

    optimizer.step()

    if ii == 5:
      break

  scheduler.step()
  print("Epoch number: {} and the loss : {}".format(i,loss.item()))

  torch.save({
            'epoch': i,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            #'loss': loss,
            }, '/kaggle/working/checkpoint.pth')




Adjusting learning rate of group 0 to 2.0000e-04.


  0%|          | 0/736 [00:00<?, ?it/s]

torch.Size([3, 256, 256])
0 0
1
torch.Size([1, 3, 256, 256])
torch.Size([1, 3, 256, 256])


  0%|          | 1/736 [00:10<2:05:00, 10.21s/it]

torch.Size([3, 256, 256])
0 1
1
torch.Size([1, 3, 256, 256])
torch.Size([1, 3, 256, 256])


...

torch.Size([3, 256, 256])
1 4
1
torch.Size([1, 3, 256, 256])
torch.Size([1, 3, 256, 256])


  1%|          | 5/736 [00:15<37:06,  3.05s/it]

torch.Size([3, 256, 256])
1 5
1
torch.Size([1, 3, 256, 256])
torch.Size([1, 3, 256, 256])


  1%|          | 5/736 [00:18<44:30,  3.65s/it]


Adjusting learning rate of group 0 to 1.9980e-04.
Epoch number: 1 and the loss : 11.16543960571289


In [33]:
checkpoint = torch.load('/kaggle/working/checkpoint.pth')
#model.load_state_dict(checkpoint['model_state_dict'])
#optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
#loss = checkpoint['loss']

In [34]:
epoch

1